# FastAPI + Machine Learning = 💖

## What you'll learn in this course 🧐🧐

FastAPI is a powerful tool to serve Machine Learning models. In this course, we will teach you: 

* How to build endpoints that serves ML models 
* Customize your code for batch predictions 


## Demo App 

For this course, we will still be using this [Demo API](https://jedha-fast-api-demo.herokuapp.com/). Feel free to `git clone` [this repository](https://github.com/JedhaBootcamp/fast_api_demo_app) if you want to check out the source code. 

## Serve ML Models 

### **`mlflow`** power 

Let's combine the power of `mlflow` with FastAPI. Let's open up the URL of one of our model in our MLFLOW Tracking application:

* [Salary Estimator](https://sample-mlflow-app.herokuapp.com/#/experiments/7)

Click on the first experiment and you should see the *Artifacts* folder with some code example: 

![snap](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/Mlflow_model_serving.png)

As you can see, it requires only a few lines of code to start using your model. All you have to do is to load it using `mlflow` and this code: 

```python
# Read data 
df = pd.DataFrame({"some": ["data"]})

# Log model from mlflow 
logged_model = 'YOUR_MLFLOW_RUN_URI'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
prediction = loaded_model.predict(df)

```
And that's it! 

In our application, we used `mlflow` in FastAPI to predict salaries given a number of years of experience just like this: 


```python 
class PredictionFeatures(BaseModel):
    YearsExperience: float

#### SOME CODE ####
###################

@app.post("/predict", tags=["Machine Learning"])
async def predict(predictionFeatures: PredictionFeatures):
    """
    Prediction of salary for a given year of experience! 
    """
    # Read data 
    years_experience = pd.DataFrame({"YearsExperience": [predictionFeatures.YearsExperience]})

    # Log model from mlflow 
    logged_model = 'runs:/323c3b4a6a6242b7837681bd5c539b27/salary_estimator'

    # Load model as a PyFuncModel.
    loaded_model = mlflow.pyfunc.load_model(logged_model)
    prediction = loaded_model.predict(years_experience)

    # Format response
    response = {"prediction": prediction.tolist()[0]}
    return response
```

As you can see, we simply loaded the data (`predictionFeatures.YearsExperience`) as a `DataFrame` and then used our model logged in our app. 


> 👋 This code will **only work if your model is logged on Mlflow Registry**. Make sure that it's the case 😉

## Run the app 

If you want to run the app, simply use Docker as usual: 

```bash 
docker run -it \
-v "$(pwd):/home/app" \
-p 4000:4000 \
-e PORT=4000 \
-e MLFLOW_TRACKING_URI=$MLFLOW_TRACKING_URI \
-e AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID \
-e AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY \
-e BACKEND_STORE_URI=$BACKEND_STORE_URI \
-e ARTIFACT_ROOT=$ARTIFACT_ROOT \
jedha/fast_api_demo_app
```

As usual, make sure you define your environment variables! 

> 👋 Your AWS credentials need to be authorized by Jedha, otherwise the API won't be able to access `mlflow`. If you don't have credentials, you will need to create your own model, with your own API in production and your own Docker image.

## Batch predictions 

The above code that we showed you **is great for one prediction** but what happens if you want to make a lot of predictions at once? Well this is what is called **batch predictions**. To do so you will need to upload a file (*i.e* `.csv` or `.xlsx`) and then make predictions. 

### Upload files with FastAPI 

First, let's learn how to upload files using FastAPI. To do so, you will need to import `File` and `UploadFile` from `fastapi`: 

```python 
from fastapi import FastAPI, File, UploadFile
### SOME CODE### 
```

Then simply create a **`POST`** request that will use this file like this:

```python 
@app.post("/post-picture", tags=["Blog Endpoints"])
async def post_picture(file: UploadFile= File(...)):
    """
    Upload a picture and read its file name.
    """
    return {"picture": file.filename}
```

Here we simply upload a file and read its name using the `file.filename` attribute. 


### Batch predictions with files 

Now `file.filename` attribute isn't really useful. However, you can use `file.file` to read the actual content of the file and therefore load it in a `pandas` DataFrame. 

```python 
@app.post("/batch-pred")
async def batch_pred(file: UploadFile = File(...)):
    """
    Make batch predictions 

    """
    df = pd.read_excel(file.file)

    # Log model from mlflow 
    logged_model = 'runs:/5e54b2ee620546b0914c9e9fbfd18875/salary_estimator'

    # Load model as a PyFuncModel.
    loaded_model = mlflow.pyfunc.load_model(logged_model)
    prediction = loaded_model.predict(df)

    # Format response
    response = {"prediction": prediction.tolist()}

    return response
```

And voilà! 

## Resources 📚📚

* [Request Body](https://fastapi.tiangolo.com/tutorial/body/)
* [Request Files](https://fastapi.tiangolo.com/tutorial/request-files/)